# Random Forest Exploration

In [1]:
# Initial imports.
import pandas as pd
from pathlib import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [2]:
# Loading data

filepath = "../Resources/City_envo_hpi.csv"
df = pd.read_csv(filepath)
df

,stateabbr,placename,placefips,tractfips,place_tractid,population2010,insurance,highbloodpressure,cancer,asthma,...,hpi,Place_TractID,SiteId,PM2.5_Exceptional,PM2.5_ExceedCount,PM2.5_Max,Ozone_Exceptional,Ozone_ExceedCount,Ozone_Max,sfcount
0,AL,Birmingham,107000,1073003200,0107000-01073003200,931,26.8,57.0,6.0,14.4,...,147.35,0107000-01073003200,11.0,0.0,0.0,26.6,0.0,1.0,0.078,0.0
1,AL,Birmingham,107000,1073003300,0107000-01073003300,947,21.4,55.2,6.8,12.9,...,211.99,0107000-01073003300,11.0,0.0,0.0,26.6,0.0,1.0,0.078,0.0
2,AL,Birmingham,107000,1073010500,0107000-01073010500,114,23.9,60.5,7.1,13.6,...,167.48,0107000-01073010500,11.0,0.0,0.0,26.6,0.0,1.0,0.078,0.0
3,AL,Birmingham,107000,1073010701,0107000-01073010701,74,19.8,24.9,2.7,11.2,...,670.86,0107000-01073010701,11.0,0.0,0.0,26.6,0.0,1.0,0.078,0.0
4,AL,Birmingham,107000,1073010801,0107000-01073010801,168,7.3,33.0,7.4,8.9,...,607.68,0107000-01073010801,11.0,0.0,0.0,26.6,0.0,1.0,0.078,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1373,WI,Racine,5566000,55101001505,5566000-55101001505,829,11.3,21.8,4.1,10.5,...,232.43,5566000-55101001505,2496.0,0.0,0.0,25.4,0.0,1.0,0.071,0.0
1374,WI,Waukesha,5584250,55133202902,5584250-55133202902,747,9.6,28.4,7.6,8.6,...,340.44,5584250-55133202902,2517.0,0.0,0.0,30.1,0.0,0.0,0.067,0.0
1375,WI,Waukesha,5584250,55133203804,5584250-55133203804,894,7.4,28.6,7.3,8.6,...,374.88,5584250-55133203804,2517.0,0.0,0.0,30.1,0.0,0.0,0.067,0.0
1376,WY,Cheyenne,5613900,56021000402,5613900-56021000402,415,23.9,27.3,4.7,10.9,...,268.00,5613900-56021000402,2595.0,0.0,0.0,14.9,0.0,0.0,0.057,1.0


In [3]:
median_hpi = df['hpi'].median()
print(median_hpi)

391.08000000000004


In [4]:
performance = []
dummy = []
for value in df['hpi']:
    if value >= median_hpi:
        performance.append('Overperforming')
        dummy.append(1)
    else:
        performance.append('Underperforming')
        dummy.append(0)
df['Performance'] = performance
df['dummy'] = dummy
df.head(10)

,stateabbr,placename,placefips,tractfips,place_tractid,population2010,insurance,highbloodpressure,cancer,asthma,...,SiteId,PM2.5_Exceptional,PM2.5_ExceedCount,PM2.5_Max,Ozone_Exceptional,Ozone_ExceedCount,Ozone_Max,sfcount,Performance,dummy
0,AL,Birmingham,107000,1073003200,0107000-01073003200,931,26.8,57.0,6.0,14.4,...,11.0,0.0,0.0,26.6,0.0,1.0,0.078,0.0,Underperforming,0
1,AL,Birmingham,107000,1073003300,0107000-01073003300,947,21.4,55.2,6.8,12.9,...,11.0,0.0,0.0,26.6,0.0,1.0,0.078,0.0,Underperforming,0
2,AL,Birmingham,107000,1073010500,0107000-01073010500,114,23.9,60.5,7.1,13.6,...,11.0,0.0,0.0,26.6,0.0,1.0,0.078,0.0,Underperforming,0
3,AL,Birmingham,107000,1073010701,0107000-01073010701,74,19.8,24.9,2.7,11.2,...,11.0,0.0,0.0,26.6,0.0,1.0,0.078,0.0,Overperforming,1
4,AL,Birmingham,107000,1073010801,0107000-01073010801,168,7.3,33.0,7.4,8.9,...,11.0,0.0,0.0,26.6,0.0,1.0,0.078,1.0,Overperforming,1
5,AL,Birmingham,107000,1073010802,0107000-01073010802,172,6.6,37.2,11.2,8.4,...,11.0,0.0,0.0,26.6,0.0,1.0,0.078,0.0,Overperforming,1
6,AL,Birmingham,107000,1073010803,0107000-01073010803,514,12.5,31.7,5.7,9.9,...,11.0,0.0,0.0,26.6,0.0,1.0,0.078,0.0,Overperforming,1
7,AL,Birmingham,107000,1073010805,0107000-01073010805,86,9.7,49.6,7.8,9.8,...,11.0,0.0,0.0,26.6,0.0,1.0,0.078,0.0,Overperforming,1
8,AL,Birmingham,107000,1073011207,0107000-01073011207,815,19.2,39.3,4.6,12.7,...,11.0,0.0,0.0,26.6,0.0,1.0,0.078,0.0,Underperforming,0
9,AL,Birmingham,107000,1073011803,0107000-01073011803,928,20.9,42.1,5.1,12.9,...,11.0,0.0,0.0,26.6,0.0,1.0,0.078,0.0,Underperforming,0


In [5]:
df = df.dropna()

In [6]:
df.dtypes

stateabbr              object
placename              object
placefips               int64
tractfips               int64
place_tractid          object
population2010          int64
insurance             float64
highbloodpressure     float64
cancer                float64
asthma                float64
heartdisease          float64
annualcheckups        float64
smokes                float64
mentalhealthissues    float64
latitude              float64
longitude             float64
countyfips              int64
State                  object
StateAbbr              object
County                 object
CountyFIPS              int64
Asthma                float64
AFHeartDisease        float64
COPD                  float64
Depression            float64
HeartFailure          float64
Hypertension          float64
CoronaryArtery        float64
Psychotic             float64
Stroke                float64
TotalCost             float64
tract                 float64
hpi                   float64
Place_Trac

In [7]:
num_df = df.copy()
num_df = num_df[["insurance", "highbloodpressure", "cancer", "asthma", 
"heartdisease", "annualcheckups",  "smokes", "mentalhealthissues",   
"sfcount", "Asthma", "AFHeartDisease",   "COPD", "Depression", 
"HeartFailure", "Hypertension", "CoronaryArtery", "Psychotic", 
"Stroke", "TotalCost", "PM2.5_ExceedCount",  "PM2.5_Max", 
"Ozone_ExceedCount",  "Ozone_Max"]]

In [10]:
cms = ["Asthma", "AFHeartDisease",   "COPD", "Depression", 
"HeartFailure", "Hypertension", "CoronaryArtery", "Psychotic", 
"Stroke", "TotalCost"]

for value in cms:
    num_df[value] = num_df[value]/1000

num_df['sfcount'] = num_df['sfcount']*10

num_df

,insurance,highbloodpressure,cancer,asthma,heartdisease,annualcheckups,smokes,mentalhealthissues,sfcount,Asthma,...,HeartFailure,Hypertension,CoronaryArtery,Psychotic,Stroke,TotalCost,PM2.5_ExceedCount,PM2.5_Max,Ozone_ExceedCount,Ozone_Max
0,26.8,57.0,6.0,14.4,11.1,78.2,29.6,22.0,0.0,0.000022,...,0.000030,0.000015,0.000021,0.000022,0.000034,0.000472,0.0,26.6,1.0,0.078
1,21.4,55.2,6.8,12.9,9.7,80.1,22.6,16.7,0.0,0.000022,...,0.000030,0.000015,0.000021,0.000022,0.000034,0.000472,0.0,26.6,1.0,0.078
2,23.9,60.5,7.1,13.6,11.1,80.7,26.5,18.6,0.0,0.000022,...,0.000030,0.000015,0.000021,0.000022,0.000034,0.000472,0.0,26.6,1.0,0.078
3,19.8,24.9,2.7,11.2,2.4,66.2,18.1,16.1,0.0,0.000022,...,0.000030,0.000015,0.000021,0.000022,0.000034,0.000472,0.0,26.6,1.0,0.078
4,7.3,33.0,7.4,8.9,4.5,73.7,8.5,8.8,10.0,0.000022,...,0.000030,0.000015,0.000021,0.000022,0.000034,0.000472,0.0,26.6,1.0,0.078
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1373,11.3,21.8,4.1,10.5,2.6,68.4,14.7,11.5,0.0,0.000022,...,0.000029,0.000015,0.000019,0.000016,0.000029,0.000433,0.0,25.4,1.0,0.071
1374,9.6,28.4,7.6,8.6,5.2,71.8,13.7,9.9,0.0,0.000020,...,0.000026,0.000013,0.000017,0.000015,0.000030,0.000405,0.0,30.1,0.0,0.067
1375,7.4,28.6,7.3,8.6,4.8,72.0,14.2,9.7,0.0,0.000020,...,0.000026,0.000013,0.000017,0.000015,0.000030,0.000405,0.0,30.1,0.0,0.067
1376,23.9,27.3,4.7,10.9,4.9,59.9,26.3,15.5,10.0,0.000018,...,0.000024,0.000014,0.000019,0.000017,0.000033,0.000392,0.0,14.9,0.0,0.057


In [11]:
# Define the features set.
X = num_df.copy()
X.describe()

,insurance,highbloodpressure,cancer,asthma,heartdisease,annualcheckups,smokes,mentalhealthissues,sfcount,Asthma,...,HeartFailure,Hypertension,CoronaryArtery,Psychotic,Stroke,TotalCost,PM2.5_ExceedCount,PM2.5_Max,Ozone_ExceedCount,Ozone_Max
count,1279.000000,1279.000000,1279.000000,1279.000000,1279.000000,1279.000000,1279.000000,1279.000000,1279.000000,1279.000000,...,1279.000000,1279.000000,1279.000000,1279.000000,1279.000000,1279.000000,1279.000000,1279.000000,1279.000000,1279.000000
mean,16.408053,31.882252,6.171228,9.529789,5.971462,68.946364,17.886396,13.630180,1.727912,0.000022,...,0.000029,0.000015,0.000020,0.000023,0.000033,0.000453,1.430023,33.988898,7.749805,0.077245
std,8.543887,10.132310,2.939231,1.642709,3.275846,6.688412,6.524669,3.778265,6.069583,0.000003,...,0.000003,0.000002,0.000002,0.000006,0.000005,0.000059,3.648427,22.542712,19.365158,0.014233
min,2.400000,5.300000,0.700000,5.600000,0.300000,48.200000,2.200000,4.200000,0.000000,0.000012,...,0.000021,0.000010,0.000012,0.000010,0.000020,0.000298,0.000000,11.800000,0.000000,0.050000
25%,10.000000,25.150000,4.200000,8.400000,3.800000,64.500000,13.300000,10.900000,0.000000,0.000020,...,0.000027,0.000014,0.000018,0.000018,0.000029,0.000407,0.000000,21.800000,0.000000,0.070000
50%,14.500000,30.100000,5.700000,9.200000,5.300000,68.500000,17.000000,13.100000,0.000000,0.000022,...,0.000029,0.000015,0.000020,0.000022,0.000033,0.000453,0.000000,27.400000,2.000000,0.076000
75%,20.350000,37.300000,7.400000,10.400000,7.400000,73.100000,21.500000,15.700000,0.000000,0.000024,...,0.000032,0.000016,0.000022,0.000027,0.000036,0.000498,1.000000,40.000000,5.000000,0.080000
max,52.900000,70.300000,22.300000,17.500000,35.800000,90.200000,50.300000,28.000000,80.000000,0.000030,...,0.000042,0.000019,0.000026,0.000036,0.000047,0.000598,29.000000,231.600000,112.000000,0.136000


In [12]:
for column in X.columns:
    print(f"Column {column} has {X[column].isnull().sum()} nul values")

Column insurance has 0 nul values
Column highbloodpressure has 0 nul values
Column cancer has 0 nul values
Column asthma has 0 nul values
Column heartdisease has 0 nul values
Column annualcheckups has 0 nul values
Column smokes has 0 nul values
Column mentalhealthissues has 0 nul values
Column sfcount has 0 nul values
Column Asthma has 0 nul values
Column AFHeartDisease has 0 nul values
Column COPD has 0 nul values
Column Depression has 0 nul values
Column HeartFailure has 0 nul values
Column Hypertension has 0 nul values
Column CoronaryArtery has 0 nul values
Column Psychotic has 0 nul values
Column Stroke has 0 nul values
Column TotalCost has 0 nul values
Column PM2.5_ExceedCount has 0 nul values
Column PM2.5_Max has 0 nul values
Column Ozone_ExceedCount has 0 nul values
Column Ozone_Max has 0 nul values


In [13]:
# Define target vector
y = df["dummy"].ravel()
y[:5]

array([0, 0, 0, 1, 1], dtype=int64)

In [14]:
# Splitting into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [15]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [16]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=78) 

In [17]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

In [18]:
# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)

In [19]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,133,37
Actual 1,34,116


In [20]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)

In [21]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,133,37
Actual 1,34,116


Accuracy Score : 0.778125
Classification Report
              precision    recall  f1-score   support

           0       0.80      0.78      0.79       170
           1       0.76      0.77      0.77       150

    accuracy                           0.78       320
   macro avg       0.78      0.78      0.78       320
weighted avg       0.78      0.78      0.78       320



In [22]:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
importances

array([0.0575536 , 0.04623626, 0.0402003 , 0.04156689, 0.0382915 ,
       0.04115232, 0.08097877, 0.04442492, 0.0055051 , 0.03623769,
       0.03721164, 0.03333151, 0.03404611, 0.03659484, 0.03716556,
       0.02987465, 0.04640271, 0.0330049 , 0.02908684, 0.0635549 ,
       0.10575088, 0.03262633, 0.04920179])

In [23]:
# We can sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.10575087692961198, 'PM2.5_Max'),
 (0.08097877095288998, 'smokes'),
 (0.06355489508188322, 'PM2.5_ExceedCount'),
 (0.05755360046783415, 'insurance'),
 (0.04920179188627155, 'Ozone_Max'),
 (0.04640270526060459, 'Psychotic'),
 (0.04623626035834815, 'highbloodpressure'),
 (0.04442491912255332, 'mentalhealthissues'),
 (0.04156689190526512, 'asthma'),
 (0.041152323574304255, 'annualcheckups'),
 (0.04020029597273756, 'cancer'),
 (0.03829149745435858, 'heartdisease'),
 (0.037211636340060805, 'AFHeartDisease'),
 (0.037165564289176375, 'Hypertension'),
 (0.036594840556638926, 'HeartFailure'),
 (0.03623769186051811, 'Asthma'),
 (0.03404611321655103, 'Depression'),
 (0.03333151009866553, 'COPD'),
 (0.03300489582153038, 'Stroke'),
 (0.03262633429203051, 'Ozone_ExceedCount'),
 (0.029874645447244448, 'CoronaryArtery'),
 (0.02908683612549136, 'TotalCost'),
 (0.005505102985429949, 'sfcount')]